In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.simplefilter("ignore")

Importando nuestros DFs Poblacion y Homicidios para generar los DFs necesarios para nuestros KPIs

In [2]:
homicidiosdf = pq.read_table("../0-DATA/homicidiosEDAunion.parquet").to_pandas()
poblaciondf = pq.read_table("../0-DATA/PoblacionBBDD.parquet").to_pandas()

In [3]:
homicidiosdf.head(3)

,Id,Victimas,Hora,Lugar,Calle,Longitud,Latitud,Acusado,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Barrio,nombre_comuna,url_imagen
0,2016-0002,1,1,GRAL PAZ,"PAZ, GRAL. AV.",-58.508775,-34.669777,PASAJEROS,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,Mataderos,"Liniers, Mataderos y Parque Avellaneda",https://buenosaires.gob.ar/sites/default/files...
1,2016-0003,1,7,AVENIDA,ENTRE RIOS AV.,-58.390403,-34.631894,AUTO,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,Constitución,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",https://buenosaires.gob.ar/sites/default/files...
2,2016-0004,1,0,AVENIDA,LARRAZABAL AV.,-58.465039,-34.680930,AUTO,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,Villa Lugano,"Villa Soldati, Villa Riachuelo y Villa Lugano",https://buenosaires.gob.ar/sites/default/files...


In [4]:
poblaciondf

,Año,Poblacion
0,1810,44800
1,1869,177797
2,1895,663854
3,1914,1575814
4,1947,2981043
5,1960,2966634
6,1970,2972453
7,1980,2922829
8,1991,2965403
9,2001,2776138


### KPI 1

Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

In [5]:
def tasahomicidioscabasemestral(homicidios, poblacion, año_inicial=2016, año_final=2022):
    datos = []
    for año in range(año_inicial, año_final):
        # Filtrar homicidios por semestre
        homicidios_JD = len(homicidios[(homicidios["Mes"].isin([7, 8, 9, 10, 11, 12])) & (homicidios["Año"] == año)])
        homicidios_EJ = len(homicidios[(homicidios["Mes"].isin([1, 2, 3, 4, 5, 6])) & (homicidios["Año"] == año)])
        
        # Obtener población del año actual
        poblacion_actual = poblacion[poblacion["Año"] == año]["Poblacion"].iloc[0]
        
        # Calcular tasas de homicidios por semestre
        tasa_JD = (homicidios_JD / poblacion_actual) * 100000
        tasa_EJ = (homicidios_EJ / poblacion_actual) * 100000
        
        # Calcular porcentaje de comparación
        porcentaje_comparacion = (tasa_JD - tasa_EJ) * 100
        
        # Agregar datos a la lista
        datos.append({
            'Año': año,
            'Homicidios_EJ': homicidios_EJ,
            'Homicidios_JD': homicidios_JD,
            'Poblacion': poblacion_actual,
            'Tasa_EJ': tasa_EJ,
            'Tasa_JD': tasa_JD,
            'Porcentaje_de_comparacion': porcentaje_comparacion
        })
    
    # Crear DataFrame a partir de la lista de datos
    df_resultado = pd.DataFrame(datos)
    
    return df_resultado

# Ejemplo de uso
# Suponiendo que tienes los DataFrames homicidios y poblacion
# homicidios = ...
# poblacion = ...

# Llamar a la función
resultadokpi1 = tasahomicidioscabasemestral(homicidiosdf, poblaciondf)

# Imprimir el resultado
resultadokpi1


,Año,Homicidios_EJ,Homicidios_JD,Poblacion,Tasa_EJ,Tasa_JD,Porcentaje_de_comparacion
0,2016,63,76,3005929,2.095858,2.528336,43.247861
1,2017,65,70,3025225,2.148601,2.313877,16.527696
2,2018,69,79,3044521,2.266366,2.594825,32.845889
3,2019,57,47,3063818,1.860424,1.534034,-32.639014
4,2020,30,49,3083114,0.973042,1.589302,61.626005
5,2021,53,41,3102410,1.708349,1.321553,-38.679607


### KPI 2

Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

In [6]:
def porcentajehomicidioscabaanual(homicidios, poblacion, año_inicial=2016, año_final=2022):
    datos = []
    for año in range(año_inicial, año_final):
        # Filtrar homicidios por semestre
        homicidios_año = len(homicidios[(homicidios["Victima"] == 'MOTO') & (homicidios["Año"] == año)])
        homicidios_añofa = len(homicidios[(homicidios["Victima"] == 'MOTO') & (homicidios["Año"] == año-1)])

        # Obtener población del año actual
        poblacion_actual = poblacion[poblacion["Año"] == año]["Poblacion"].iloc[0]
        # Calcular tasas de homicidios por semestre
        tasa_año = (homicidios_año / poblacion_actual) * 100000
        
        # Calcular porcentaje de comparación
        if homicidios_añofa == 0:
            porcentaje_comparacion = "0.0"
        else:
            porcentaje_comparacion = ((-homicidios_añofa + homicidios_año)/homicidios_añofa) * 100
        # Agregar datos a la lista
        datos.append({
            'Año': año,
            'Homicidios_Año_actual': homicidios_año,
            'Tasa_Año_actual': tasa_año,
            'Porcentaje_de_comparacion': porcentaje_comparacion
        })

    # Crear DataFrame a partir de la lista de datos
    df_resultado = pd.DataFrame(datos)
    
    return df_resultado

# Ejemplo de uso
# Suponiendo que tienes los DataFrames homicidios y poblacion
# homicidios = ...
# poblacion = ...

# Llamar a la función
resultadokpi2 = porcentajehomicidioscabaanual(homicidiosdf, poblaciondf)

# Imprimir el resultado
resultadokpi2


,Año,Homicidios_Año_actual,Tasa_Año_actual,Porcentaje_de_comparacion
0,2016,65,2.162393,0.0
1,2017,56,1.851102,-13.846154
2,2018,57,1.872216,1.785714
3,2019,50,1.631951,-12.280702
4,2020,28,0.908173,-44.0
5,2021,44,1.418252,57.142857


### KPI 3

Analizaremos la tasa de accidentes por género

In [7]:
def tasahomicidioscabasexo(homicidios, poblacion, año_inicial=2016, año_final=2022):
    datos = []
    for año in range(año_inicial, año_final):
        # Filtrar homicidios por semestre
        homicidios_añoFE = len(homicidios[(homicidios["Sexo"] == 'FEMENINO') & (homicidios["Año"] == año)])
        homicidios_añofaFE = len(homicidios[(homicidios["Sexo"] == 'FEMENINO') & (homicidios["Año"] == año-1)])
        homicidios_añoMA = len(homicidios[(homicidios["Sexo"] == 'MASCULINO') & (homicidios["Año"] == año)])
        homicidios_añofaMA = len(homicidios[(homicidios["Sexo"] == 'MASCULINO') & (homicidios["Año"] == año-1)])

        # Obtener población del año actual
        poblacion_actual = poblacion[poblacion["Año"] == año]["Poblacion"].iloc[0]
        poblacion_fa = poblacion[poblacion["Año"] == año-1]["Poblacion"].iloc[0]
        # Calcular tasas de homicidios por semestre
        tasa_añoFE = (homicidios_añoFE / poblacion_actual) * 100000
        tasa_añofaFE = (homicidios_añofaFE / poblacion_fa) * 100000
        tasa_añoMA = (homicidios_añoMA / poblacion_actual) * 100000
        tasa_añofaMA = (homicidios_añofaMA / poblacion_fa) * 100000
        # Calcular porcentaje de comparación
        if tasa_añofaFE == 0 or tasa_añofaMA == 0:
            porcentaje_comparacionFE = "0.0"
            porcentaje_comparacionMA = "0.0"
        else:
            porcentaje_comparacionFE = ((tasa_añoFE - tasa_añofaFE)/tasa_añofaFE) * 100
            porcentaje_comparacionMA = ((tasa_añoMA - tasa_añofaMA)/tasa_añofaMA) * 100

        # Agregar datos a la lista
        datos.append({
            'Año': año,
            'Homicidios_Año_actual_FE': homicidios_añoFE,
            'Tasa_Año_actual_FE': tasa_añoFE,
            'Porcentaje_de_cambio_FE': porcentaje_comparacionFE,
            'Homicidios_Año_actual_MA': homicidios_añoMA,
            'Tasa_Año_actual_MA': tasa_añoMA,
            'Porcentaje_de_cambio_MA': porcentaje_comparacionMA
        })

    # Crear DataFrame a partir de la lista de datos
    df_resultado = pd.DataFrame(datos)
    
    return df_resultado

# Ejemplo de uso
# Suponiendo que tienes los DataFrames homicidios y poblacion
# homicidios = ...
# poblacion = ...

# Llamar a la función
resultadokpi3 = tasahomicidioscabasexo(homicidiosdf, poblaciondf)

# Imprimir el resultado
resultadokpi3

,Año,Homicidios_Año_actual_FE,Tasa_Año_actual_FE,Porcentaje_de_cambio_FE,Homicidios_Año_actual_MA,Tasa_Año_actual_MA,Porcentaje_de_cambio_MA
0,2016,27,0.898225,0.0,112,3.725970,0.0
1,2017,36,1.189994,32.482884,99,3.272484,-12.170945
2,2018,38,1.248144,4.88655,110,3.613048,10.406895
3,2019,27,0.881253,-29.394883,77,2.513204,-30.440885
4,2020,16,0.518956,-41.111621,63,2.043389,-18.693886
5,2021,22,0.709126,36.644794,72,2.320776,13.574894


In [12]:
#guardamos en un CSV
resultadokpi1.to_csv("../0-DATA/resultadokpi1.csv", index=False, encoding="utf-8") 
resultadokpi2.to_csv("../0-DATA/resultadokpi2.csv", index=False, encoding="utf-8") 
resultadokpi3.to_csv("../0-DATA/resultadokpi3.csv", index=False, encoding="utf-8") 

In [13]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl1 = pd.read_csv("../0-DATA/resultadokpi1.csv") 
tabla = pa.Table.from_pandas(itemsl1) 
pq.write_table(tabla,"../0-DATA/resultadokpi1.parquet") 

itemsl2 = pd.read_csv("../0-DATA/resultadokpi2.csv") 
tabla = pa.Table.from_pandas(itemsl2) 
pq.write_table(tabla,"../0-DATA/resultadokpi2.parquet") 

itemsl3 = pd.read_csv("../0-DATA/resultadokpi3.csv") 
tabla = pa.Table.from_pandas(itemsl3) 
pq.write_table(tabla,"../0-DATA/resultadokpi3.parquet") 